The `Maybe` container is used when a series of computations could return `None` at any point.

In [1]:
from typing import Any

from returns.maybe import Maybe, Some, Nothing, maybe

import toolz.curried as toolz

The `from_optional()` factory method returns one of two types: `Some` and `Nothing`.

In [2]:
assert str(Maybe.from_optional(1)) == '<Some: 1>'
assert str(Maybe.from_optional(None)) == '<Nothing>'

The `from_value()` factory method behaves a bit differently allowing its returned value to be `Some(None)`.

In [3]:
assert str(Maybe.from_value(1)) == '<Some: 1>'
assert str(Maybe.from_value(None)) == '<Some: None>'

In [4]:
from attr import dataclass
from typing import Optional

In [5]:
@dataclass
class Address:
    street: Optional[str]

@dataclass
class User:
    address: Optional[Address]

@dataclass
class Order:
    user: Optional[User]

In [6]:
def get_street_address(order: Order) -> Maybe[str]:
    return Maybe.from_optional(order.user)\
        .bind_optional(lambda user: user.address)\
        .bind_optional(lambda address: address.street)

In [7]:
with_address = Order(User(Address('9999 Some street')))
empty_user = Order(None)
empty_address = Order(User(None))
empty_street = Order(User(Address(None)))

In [8]:
str(get_street_address(with_address))

'<Some: 9999 Some street>'

In [9]:
assert get_street_address(empty_user) == Nothing
assert get_street_address(empty_address) == Nothing
assert get_street_address(empty_street) == Nothing

The `maybe` decorator

In [10]:
@maybe
def number(num: int) -> Optional[int]:
    if num > 0:
        return num
    else:
        return None

result: Maybe[int] = number(1)
assert result == Some(1)

Why `Maybe` does not have an `alt` method?

Well, because `Maybe` only has a single failed value: `Nothing` and it cannot be altered.

But, `Maybe` has a `or_else_call()` method to invoke a callback function with no arguments on a `Nothing`
container.

This method is unique to the `Maybe` container.

In [13]:
assert Some(1).or_else_call(lambda: 2) == 1
assert Nothing.or_else_call(lambda: 2) == 2